In [1]:
using LinearAlgebra

In [2]:
struct Parm
    γg::Float64
    γl::Float64
    gt::Float64
    ω::Float64
    Nmax::Int
    rmax::Float64
end

In [3]:
p0 = Parm(0.5, 1.0, 0.01, 0.1, 20, 16.0)

Parm(0.5, 1.0, 0.01, 0.1, 20, 16.0)

In [4]:
function make_GKSL(p::Parm)
    GKSL = zeros(ComplexF64, (p.Nmax+1)^2, (p.Nmax+1)^2)
    #Diagonal part
    for i in 0:p.Nmax
        for j in 0:p.Nmax
            ##consider the term act on \rho[i,j]
            #Hermitian part
            GKSL[i*(p.Nmax+1)+j+1, i*(p.Nmax+1)+j+1] += -1.0im * (i-j) * p.ω

            #loss term
            if(i>0 && j>0)
                GKSL[(i-1)*(p.Nmax+1)+j, i*(p.Nmax+1)+j+1] += p.γl * sqrt(i*j)
            end
            GKSL[i*(p.Nmax+1)+j+1, i*(p.Nmax+1)+j+1] -= (i+j) * p.γl / 2.0

            #gain term
            if(i<p.Nmax && j<p.Nmax)
                GKSL[(i+1)*(p.Nmax+1)+j+2, i*(p.Nmax+1)+j+1] += p.γg * sqrt((i+1)*(j+1))
            end
            GKSL[i*(p.Nmax+1)+j+1, i*(p.Nmax+1)+j+1] -= (i+j+2) * p.γg / 2.0

            #interaction term
            if(i>1 && j>1)
                GKSL[(i-2)*(p.Nmax+1)+j-1, i*(p.Nmax+1)+j+1] += p.gt * sqrt(i*(i-1)*j*(j-1)) /2.0
            elseif(i>1)
                GKSL[(i-2)*(p.Nmax+1)+j+1, i*(p.Nmax+1)+j+1] += p.gt * j *sqrt(i*(i-1)) /2.0 - p.gt * (i-2)*sqrt(i*(i-1)) /4.0
            elseif(j>1)
                GKSL[i*(p.Nmax+1)+j-1, i*(p.Nmax+1)+j+1] += p.gt * i *sqrt(j*(j-1)) /2.0 - p.gt * (j-2)*sqrt(j*(j-1)) /4.0
            end

            if(i<p.Nmax-1)
                GKSL[(i+2)*(p.Nmax+1)+j+1, i*(p.Nmax+1)+j+1] += p.gt * i *sqrt((i+2)*(i+1)) /4.0
            end
            if(j<p.Nmax-1)
                GKSL[i*(p.Nmax+1)+j+3, i*(p.Nmax+1)+j+1] += p.gt * j *sqrt((j+2)*(j+1)) /4.0
            end
            GKSL[i*(p.Nmax+1)+j+1, i*(p.Nmax+1)+j+1] -= (i+j+2) * p.gt / 2.0
        end
    end
    return GKSL
end

make_GKSL (generic function with 1 method)